# LlamaIndex と AgentCore Memory - 投資ポートフォリオアドバイザー（長期記憶）

## はじめに

このノートブックでは、Amazon Bedrock AgentCore Memory 機能と LlamaIndex を統合して、**長期記憶**を持つ投資ポートフォリオアドバイザーを作成する方法を紹介します。複数のクライアントミーティングや市場サイクル全体での永続性を実現し、アドバイザーが累積的な投資知識を構築し、数ヶ月から数年にわたるポートフォリオパフォーマンスを追跡できるようにします。

## アーキテクチャ概要

![LlamaIndex AgentCore Long-Term Memory Architecture](LlamaIndex-AgentCore-LTM-Arch.png)

## チュートリアルの詳細

**チュートリアル詳細:**
- **チュートリアルタイプ**: 長期クロスセッション Memory
- **エージェントユースケース**: 投資ポートフォリオアドバイザー
- **エージェントフレームワーク**: LlamaIndex
- **LLM モデル**: Anthropic Claude 3.7 Sonnet
- **チュートリアルコンポーネント**: AgentCore 長期 Memory、LlamaIndex Agent、金融ツール
- **難易度**: 上級

## ビジネス価値

**エンタープライズ投資インテリジェンス**: ポートフォリオ知識を蓄積し、投資の進化を追跡し、四半期と年にわたる包括的な市場分析を維持する永続的な AI メモリで資産運用実践を変革します。

**主な専門的利点:**
- **ポートフォリオの継続性**: 投資期間とチームメンバー間のシームレスな知識移転
- **投資記憶**: 重要な市場インサイト、戦略、パフォーマンスデータを永続的に保存
- **クロスポートフォリオインテリジェンス**: 複数のクライアントポートフォリオにわたるパターンと関連性を特定
- **戦略的卓越性**: 優れた投資決定のための過去のパフォーマンスデータを活用
- **クライアントリレーションシップ**: 複数年にわたる資産運用の詳細なコンテキストを維持
- **リスク管理**: 市場サイクルと投資戦略への影響を追跡

## 長期記憶の設定

**技術セットアップ**: このチュートリアルでは、12ヶ月保持の Semantic Strategy を使用した AgentCore Memory を使用します：
- **Memory タイプ**: 自動インサイト抽出を備えた Semantic Strategy
- **保持期間**: ポートフォリオの継続性のための365日イベント有効期限
- **クロスセッション**: 同じ actor_id + memory_id、投資期間ごとに異なる session_id
- **検索機能**: ポートフォリオ履歴全体のセマンティック検索のための組み込みメモリ取得ツール

## 技術概要

**主な長期記憶コンポーネント:**
1. **Semantic Strategy 設定**: 365日保持の自動インサイト抽出用 SemanticStrategy を使用
2. **クロスセッション永続性**: 同じ actor_id + memory_id、期間ごとに異なる session_id で知識の継続性を実現
3. **カスタム Memory 検索ツール**: AgentCore のネイティブ search_long_term_memories() を LlamaIndex FunctionTool でラップ
4. **セマンティック処理パイプライン**: 会話イベント → セマンティックメモリ変換のための90秒待機
5. **動的セッション管理**: 柔軟なセッション処理のため memory.context.session_id を使用

**学習内容:**

- 複数のクライアントミーティングにわたる永続的な AgentCore Memory の作成
- 時間経過に伴う累積的な投資知識の構築
- 市場調査とクライアント履歴全体のセマンティック検索の実装
- ポートフォリオの進化と投資パフォーマンスを追跡
- クロスセッション金融知識の永続性と取得のテスト

## シナリオの背景

この例では、四半期と年にわたる複数のミーティング全体でクライアント投資知識を維持する「投資ポートフォリオアドバイザー」を作成します。アドバイザーは AgentCore Memory を使用して、クライアントプロファイル、ポートフォリオパフォーマンス、市場インサイト、投資結果の永続的な知識ベースを構築し、時間とともに成長・進化させ、洗練された縦断的資産運用を可能にします。

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory 権限を持つ AWS IAM ロール：
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock モデルへのアクセス

## ステップ 1: 依存関係のインストールとセットアップ

In [ ]:
# Install necessary libraries including semantic strategy toolkit
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3 bedrock-agentcore-starter-toolkit

In [ ]:
# Import required components
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.semantic import SemanticStrategy
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

print("✅ All dependencies imported successfully!")

## ステップ 2: AgentCore Memory の設定

長期投資知識のための AgentCore Memory リソースを作成または取得します：

In [ ]:
# Create AgentCore Memory with Semantic Strategy for long-term persistence
region = os.getenv('AWS_REGION', 'us-east-1')
memory_manager = MemoryManager(region_name=region)

try:
    # Create memory with semantic strategy for automatic insight extraction
    memory = memory_manager.get_or_create_memory(
        name=f'InvestmentAdvisorSemantic_{int(datetime.now().timestamp())}',
        strategies=[SemanticStrategy(name="investmentLongTermMemory")],
        event_expiry_days=365  # 12-month retention for investment records
    )
    memory_id = memory.get('id')
    print(f"✅ Created Semantic Memory: {memory_id}")
    print(f"   Status: {memory.get('status')}")
    print(f"   Strategies: {[s.get('name') if isinstance(s, dict) else str(s) for s in memory.get('strategies', [])]}")
    
    # Wait for memory to become ACTIVE
    if memory.get('status') != 'ACTIVE':
        print(f"\n⏳ Waiting for memory to become ACTIVE (currently {memory.get('status')})...")
        import time
        max_wait = 300  # 5 minutes max
        waited = 0
        while waited < max_wait:
            time.sleep(10)
            waited += 10
            # Check status
            current_memory = memory_manager.get_memory(memory_id)
            status = current_memory.get('status')
            print(f"   [{waited}s] Status: {status}")
            if status == 'ACTIVE':
                print(f"✅ Memory is now ACTIVE! (took {waited} seconds)")
                break
        else:
            print(f"⚠️  Memory still not ACTIVE after {max_wait}s. Proceeding anyway...")
    
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # Replace with existing memory ID

## ステップ 3: 投資ツールの実装

縦断的資産運用のための専門ツールを定義します：

In [ ]:
def record_client_meeting(client_id: str, meeting_type: str, portfolio_value: str, key_decisions: str) -> str:
    """Record client meeting with portfolio updates and decisions"""
    return f"📅 Recorded {meeting_type} for {client_id} (${portfolio_value})"

def track_portfolio_performance(client_id: str, period: str, return_pct: str, benchmark_return: str, attribution: str) -> str:
    """Track portfolio performance vs benchmark with attribution analysis"""
    return f"📈 {client_id} {period}: {return_pct} vs {benchmark_return}"

def document_market_insight(insight_type: str, market_event: str, impact_assessment: str, client_implications: str) -> str:
    """Document market insight with client portfolio implications"""
    print(f"🌍 Market insight: {insight_type} - {market_event} (Impact: {impact_assessment})")
    return f"Documented market insight: {insight_type}"

def update_investment_thesis(client_id: str, asset_class: str, thesis: str, conviction_level: str) -> str:
    """Update investment thesis for specific asset class"""
    print(f"💭 Investment thesis: {client_id} - {asset_class} ({conviction_level} conviction)")
    return f"Updated thesis for {client_id}"

def log_rebalancing_action(client_id: str, action_type: str, securities: str, rationale: str) -> str:
    """Log portfolio rebalancing actions with rationale"""
    print(f"⚖️ Rebalancing: {client_id} - {action_type}: {securities}")
    return f"Logged rebalancing for {client_id}"

def log_advisory_milestone(quarter: str, milestone: str, details: str) -> str:
    """Log an advisory milestone with quarter and detailed progress"""
    print(f"🎯 {quarter} milestone: {milestone}")
    return f"Logged milestone for {quarter}: {milestone} - {details}"

def track_investment_metrics(metric_type: str, value: str, client_id: str, quarter: str) -> str:
    """Track specific investment metrics with client and timeline"""
    print(f"📊 {quarter}: {metric_type} = {value} (for {client_id})")
    return f"Tracked {metric_type}: {value} for {client_id} in {quarter}"

def save_advisory_insight(insight: str, quarter: str, market_context: str) -> str:
    """Save advisory insights with market context"""
    print(f"💡 {quarter} insight: {insight[:50]}...")
    return f"Saved {quarter} insight with market context: {market_context}"

# Create tool objects for the agent
investment_tools = [
    FunctionTool.from_defaults(fn=record_client_meeting),
    FunctionTool.from_defaults(fn=track_portfolio_performance),
    FunctionTool.from_defaults(fn=document_market_insight),
    FunctionTool.from_defaults(fn=update_investment_thesis),
    FunctionTool.from_defaults(fn=log_rebalancing_action),
    FunctionTool.from_defaults(fn=log_advisory_milestone),
    FunctionTool.from_defaults(fn=track_investment_metrics),
    FunctionTool.from_defaults(fn=save_advisory_insight)
]

print("✅ Investment tools created!")

## ステップ 3b: Memory 取得ツールの追加

エージェントが長期記憶を検索できるツールを作成します：

In [ ]:
def create_memory_retrieval_tool(memory_id: str, actor_id: str, region: str):
    """Create a tool for the agent to search its own long-term memory"""
    
    def search_long_term_memory(query: str) -> str:
        """Search long-term memory for relevant information about clients, portfolios, past decisions, and market insights.
        
        Use this tool when you need to recall:
        - Client information (portfolio values, risk profiles, investment goals)
        - Past investment decisions and their outcomes
        - Portfolio performance history
        - Market insights and their applications
        - Investment theses and their evolution
        
        Args:
            query: Search query describing what information you need (e.g., 'CLIENT-001 portfolio', 'investment theses', 'Q1 performance')
        
        Returns:
            Relevant information from long-term memory
        """
        try:
            from bedrock_agentcore.memory.session import MemorySessionManager
            
            # Create session manager (only needs memory_id and region)
            session_manager = MemorySessionManager(
                memory_id=memory_id,
                region_name=region
            )
            
            # Search long-term memories in the semantic strategy namespace
            results = session_manager.search_long_term_memories(
                query=query,
                namespace_prefix="/strategies",  # Search in semantic strategy namespace
                top_k=5,
                max_results=10
            )
            
            if not results:
                return "No relevant information found in long-term memory. This might be new information or the memory extraction may still be processing."
            
            # Format results for the agent
            output = "📚 Retrieved from long-term memory:\\n\\n"
            for i, result in enumerate(results, 1):
                # MemoryRecord object - access content attribute
                content = getattr(result, 'content', str(result))
                # Truncate very long content
                if len(content) > 300:
                    content = content[:300] + "..."
                output += f"{i}. {content}\\n\\n"
            
            return output
            
        except Exception as e:
            return f"⚠️ Error searching memory: {str(e)}. Proceeding without historical context."
    
    return FunctionTool.from_defaults(fn=search_long_term_memory)

# Create the memory retrieval tool
memory_search_tool = create_memory_retrieval_tool(memory_id, "financial-advisor", region)

# Add memory search to the tools list
investment_tools_with_memory = investment_tools + [memory_search_tool]

print(f"✅ Memory retrieval tool created! Total tools: {len(investment_tools_with_memory)}")
print("   Using namespace: /strategies (for semantic strategy compatibility)")


## ステップ 3c: Memory 設定の確認

Semantic Strategy が適切に設定されていることを確認します：

In [ ]:
# Check memory configuration
memory_info = memory_manager.get_memory(memory_id)
print(f"ストラテジー: {memory_info.get('strategies')}")
print(f"ステータス: {memory_info.get('status')}")
print(f"名前: {memory_info.get('name')}")

# Show strategy details
strategies = memory_info.get('strategies', [])
for strategy in strategies:
    print(f"\nストラテジー詳細:")
    print(f"  名前: {strategy.get('name')}")
    print(f"  タイプ: {strategy.get('type')}")
    print(f"  ステータス: {strategy.get('status')}")
    print(f"  ID: {strategy.get('strategyId')}")

## ステップ 4: マルチセッションエージェントの実装

異なるアドバイザリー期間をシミュレートするためのヘルパー関数を作成します：

In [ ]:
# Configuration for LONG-TERM memory (cross-session)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
ADVISOR_ID = "financial-advisor"  # Same advisor across all sessions

def create_advisory_session(session_name: str):
    """Create a new advisory session with long-term memory persistence"""
    context = AgentCoreMemoryContext(
        actor_id=ADVISOR_ID,         # Same advisor
        memory_id=memory_id,         # Same memory store (enables long-term memory)
        session_id=f"advisory-{session_name}", # Different session per period
        namespace="/wealth-management"
    )
    
    memory = AgentCoreMemory(context=context)
    llm = BedrockConverse(model=MODEL_ID)
    agent = FunctionAgent(
        tools=investment_tools_with_memory,  # Use tools with memory search capability
        llm=llm, 
        verbose=True,  # Enable verbose to see when memory is searched
        system_prompt="""You are a senior investment advisor with access to long-term memory.
        
CRITICAL: When asked about clients, portfolios, past decisions, or historical information, 
you MUST use the search_long_term_memory tool FIRST before responding.

For example:
- "What clients am I managing?" → Use search_long_term_memory("clients portfolio")
- "What was CLIENT-001's performance?" → Use search_long_term_memory("CLIENT-001 performance")
- "What investment theses do I have?" → Use search_long_term_memory("investment thesis")

Always provide conclusive, complete responses without asking follow-up questions.\n
Execute all requested actions immediately and completely. Provide detailed, professional responses."""
    )
    
    return agent, memory

print("✅ Multi-session Investment Portfolio Advisor setup complete!")


## ステップ 5: Q1 アドバイザリーセッション - 初期クライアントオンボーディング

最初のアドバイザリーセッションを開始し、クライアントのベースラインを確立します：

In [ ]:
# === Q1 ADVISORY SESSION ===
print("🗓️ === Q1: INITIAL CLIENT ONBOARDING ===")

agent_q1, memory_q1 = create_advisory_session("q1")

# Record initial client meeting
response = await agent_q1.run(
    "I'm Senior Advisor Jennifer Walsh. Record client meeting for 'CLIENT-001' with meeting type 'Initial Portfolio Review', "
    "portfolio value '$3,200,000', key decisions 'established moderate-aggressive risk profile, 20-year investment horizon, "
    "target allocation 70% equity/25% fixed income/5% alternatives'.",
    memory=memory_q1
)

print("🎯 Q1 Initial Meeting:")
print(response)

In [ ]:
# Document initial investment thesis
response = await agent_q1.run(
    "Update investment thesis for 'CLIENT-001': asset class 'US Large Cap Equity', "
    "thesis 'overweight growth stocks due to technological innovation and earnings momentum', conviction level 'high'.",
    memory=memory_q1
)
print("💭 Q1 Equity Thesis:", response)

response = await agent_q1.run(
    "Update investment thesis for 'CLIENT-001': asset class 'Fixed Income', "
    "thesis 'short duration bias due to rising rate environment, focus on credit quality', conviction level 'medium'.",
    memory=memory_q1
)
print("💭 Q1 Bond Thesis:", response)

In [ ]:
# Track initial performance baseline
response = await agent_q1.run(
    "Track portfolio performance for 'CLIENT-001': period 'Q1 2024', return_pct '+8.2%', "
    "benchmark_return '+7.1%', attribution 'tech overweight +0.8%, duration underweight +0.3%'.",
    memory=memory_q1
)
print("📈 Q1 Performance:", response)

# Verify events were stored
print("\n🔍 Verifying Q1 events were stored...")
try:
    client = MemoryClient(region_name=region)
    events = client.list_events(
        memory_id=memory_id,
        actor_id=ADVISOR_ID,
        session_id=memory_q1.context.session_id
    )
    print(f"✅ Stored {len(events)} conversational events in Q1 session")
except Exception as e:
    print(f"⚠️  Could not verify events: {e}")

# Allow time for semantic memory processing
import asyncio
print("\n⏳ Waiting for semantic memory extraction and indexing...")
print("   (AgentCore processes conversational events in the background)")
await asyncio.sleep(90)  # Increased wait time for memory extraction (was 10 seconds)
print("✅ Memory processing complete - memories should now be searchable")

## ステップ 6: Q2 アドバイザリーセッション - 市場ボラティリティへの対応

長期記憶の取得をテストし、市場の変化に適応します：

In [ ]:
# === Q2 ADVISORY SESSION ===
print("\n🗓️ === Q2: MARKET VOLATILITY RESPONSE (NEW SESSION) ===")

agent_q2, memory_q2 = create_advisory_session("q2")

# Test cross-session client recall - agent should use search_long_term_memory tool
print("\n🧠 Testing memory retrieval across sessions...")
print("   (Watch for the agent to use search_long_term_memory tool)\n")

response = await agent_q2.run(
    "What clients am I managing? What are their portfolio values, risk profiles, and investment theses?",
    memory=memory_q2
)

print("\n🧠 Q2 Client Recall:")
print(response)
print("\n✅ Expected: CLIENT-001, $3.2M portfolio, moderate-aggressive, growth equity thesis")

In [ ]:
# Document market volatility insight
response = await agent_q2.run(
    "Document market insight: insight type 'Geopolitical Risk', market event 'Trade tensions escalation', "
    "impact assessment 'increased volatility, sector rotation from growth to value', "
    "client implications 'review tech overweight, consider defensive positioning'.",
    memory=memory_q2
)
print("🌍 Q2 Market Insight:", response)

# Log rebalancing response
response = await agent_q2.run(
    "Log rebalancing action for 'CLIENT-001': action type 'Tactical Adjustment', "
    "securities 'reduced QQQ by 3%, increased VTV (value ETF) by 2%, added VGSH (short treasury) by 1%', "
    "rationale 'defensive positioning due to geopolitical uncertainty, maintain long-term allocation targets'.",
    memory=memory_q2
)
print("⚖️ Q2 Rebalancing:", response)

In [ ]:
# Track Q2 performance impact
response = await agent_q2.run(
    "Track portfolio performance for 'CLIENT-001': period 'Q2 2024', return_pct '-2.1%', "
    "benchmark_return '-3.8%', attribution 'defensive positioning +1.2%, value tilt +0.5%'.",
    memory=memory_q2
)
print("📈 Q2 Performance:", response)

# Test performance comparison
response = await agent_q2.run(
    "How did CLIENT-001's Q2 performance compare to Q1? What was the cumulative return and attribution?",
    memory=memory_q2
)
print("📊 Q2 Performance Analysis:")
print(response)
print("\n✅ Expected: Q1 +8.2%, Q2 -2.1%, cumulative ~+5.9%, defensive positioning helped")

## ステップ 7: Q3 アドバイザリーセッション - 回復と投資方針の更新

市場回復への対応と投資アプローチの更新に進みます：

In [ ]:
# === Q3 ADVISORY SESSION ===
print("\n🗓️ === Q3: MARKET RECOVERY AND THESIS UPDATE ===")

agent_q3, memory_q3 = create_advisory_session("q3")

# Record quarterly review meeting
response = await agent_q3.run(
    "Record client meeting for 'CLIENT-001' with meeting type 'Quarterly Review', "
    "portfolio value '$3,450,000', key decisions 'market recovery positioning, increase growth allocation, "
    "add international exposure for diversification'.",
    memory=memory_q3
)
print("📅 Q3 Quarterly Review:", response)

# Update investment thesis based on market evolution
response = await agent_q3.run(
    "Update investment thesis for 'CLIENT-001': asset class 'International Equity', "
    "thesis 'add developed market exposure via VTIAX, emerging markets recovery potential', conviction level 'medium'.",
    memory=memory_q3
)
print("💭 Q3 International Thesis:", response)

In [ ]:
# Test comprehensive investment history recall
response = await agent_q3.run(
    "What is the complete investment history for CLIENT-001? Include all meetings, performance periods, "
    "rebalancing actions, and evolution of investment theses.",
    memory=memory_q3
)
print("📋 Q3 Complete History:")
print(response)
print("\n✅ Expected: Q1 onboarding → Q2 defensive moves → Q3 recovery positioning, all performance data")

## ステップ 8: Q4 アドバイザリーセッション - 年末レビューと計画

セマンティック検索と年間パフォーマンス分析をテストします：

In [ ]:
# === Q4 ADVISORY SESSION ===
print("\n🗓️ === Q4: YEAR-END REVIEW AND PLANNING ===")

agent_q4, memory_q4 = create_advisory_session("q4")

# Track annual performance
response = await agent_q4.run(
    "Track portfolio performance for 'CLIENT-001': period '2024 Annual', return_pct '+12.8%', "
    "benchmark_return '+11.2%', attribution 'tactical positioning +1.1%, sector allocation +0.5%'.",
    memory=memory_q4
)
print("📈 Q4 Annual Performance:", response)

# Test market insight correlation
response = await agent_q4.run(
    "What market insights have I documented this year? How did they impact CLIENT-001's portfolio decisions?",
    memory=memory_q4
)
print("🌍 Q4 Market Insight Analysis:")
print(response)
print("\n✅ Expected: Geopolitical risk insight → defensive positioning → outperformance during volatility")

In [ ]:
# Test semantic search for similar portfolio actions
response = await agent_q4.run(
    "What rebalancing actions have I taken for CLIENT-001? Which were most effective based on subsequent performance?",
    memory=memory_q4
)
print("⚖️ Q4 Rebalancing Analysis:")
print(response)
print("\n✅ Expected: Q2 defensive moves (QQQ reduction, VTV/VGSH adds) helped during volatility")

## ステップ 9: 2年目 Q1 セッション - 複数年にわたる視点

長期投資知識とクライアント関係の進化をテストします：

In [ ]:
# === YEAR 2 Q1 ADVISORY SESSION ===
print("\n🗓️ === YEAR 2 Q1: MULTI-YEAR PERSPECTIVE ===")

agent_y2q1, memory_y2q1 = create_advisory_session("year2-q1")

# Multi-year portfolio analysis
response = await agent_y2q1.run(
    "Analyze CLIENT-001's investment journey: How has their portfolio evolved over the past year? "
    "What were the key decisions and their outcomes?",
    memory=memory_y2q1
)
print("📊 Year 2 Q1 Journey Analysis:")
print(response)
print("\n✅ Expected: $3.2M → $3.45M growth, defensive positioning success, thesis evolution")

In [ ]:
# Test investment thesis evolution tracking
response = await agent_y2q1.run(
    "How have my investment theses for CLIENT-001 evolved? What asset classes have I added and why?",
    memory=memory_y2q1
)
print("💭 Year 2 Q1 Thesis Evolution:")
print(response)
print("\n✅ Expected: Started with US equity/fixed income → added international exposure → evolved based on market conditions")

## ステップ 10: 最終資産運用ポートフォリオ評価

長期投資アドバイザリー機能の包括的なテスト：

In [ ]:
# Final comprehensive wealth management portfolio query
response = await agent_y2q1.run(
    "Provide my complete wealth management portfolio: all clients with their investment journeys, "
    "performance attribution, market insights applied, rebalancing effectiveness, and thesis evolution. "
    "Include lessons learned and best practices developed.",
    memory=memory_y2q1
)
print("💼 Complete Wealth Management Portfolio:")
print(response)
print("\n✅ Expected: Full CLIENT-001 journey with performance attribution, market timing, and investment evolution")

## 自動テスト検証
これらのセルを実行して、メモリ統合が正しく動作していることを検証します：

In [ ]:
# Define validation functions inline
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """Check if agent can recall information from earlier in the session"""
        # Check for substantive response (not just "I don't know")
        has_content = len(response) > 50
        # Check for memory indicators
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """Check if agent maintains context within session"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """Check if agent can connect current query to previous context"""
        # Look for connecting language
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# Run all validation tests
test_results = {}

# Test 1: Memory recall - can the agent recall what was discussed?
response1 = await agent_y2q1.run("What have we discussed so far in this session?", memory=memory_y2q1)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"レスポンス 1 の長さ: {len(str(response1))} 文字")

# Test 2: Session memory - does the agent maintain context?
response2 = await agent_y2q1.run("What did we talk about earlier?", memory=memory_y2q1)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"レスポンス 2 の長さ: {len(str(response2))} 文字")

# Test 3: Cross-reference capability - can it connect to previous context?
response3 = await agent_y2q1.run("How does this relate to what we discussed before?", memory=memory_y2q1)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"レスポンス 3 の長さ: {len(str(response3))} 文字")

# Display results
validator.run_validation_summary(test_results)

## まとめ

このノートブックでは、以下を実演しました：

- **長期記憶の統合**: AgentCore Memory と LlamaIndex を使用したクロスセッション資産運用

- **投資ジャーニーの追跡**: 複数の四半期にわたるポートフォリオの進化とパフォーマンスアトリビューション

- **市場インテリジェンス**: 市場インサイトとそのポートフォリオ適用のセマンティック取得

- **投資方針の進化**: 初期ポジショニングから市場適応型戦略への自然な進行

- **パフォーマンスアトリビューション**: 戦術的決定とその投資結果の詳細な追跡

- **資産運用の卓越性**: 時間をかけた包括的なクライアント関係とポートフォリオ最適化

投資ポートフォリオアドバイザーは、長期記憶がアドバイザーを時間とともに賢くなる永続的な資産運用パートナーに変換し、完全な投資履歴を維持し、長期クライアント関係全体で洗練された金融知識の取得を可能にする方法を示しています。

## クリーンアップ

このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう：

**注意**: メモリを完全に削除する場合のみ実行してください。memory_id 変数には、このノートブックで先ほど作成したメモリの ID が含まれている必要があります。

In [ ]:
# Clean up AgentCore Memory resource
try:
    from bedrock_agentcore.memory import MemoryClient
    
    client = MemoryClient(region_name=region)
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
    
except NameError as e:
    print(f"⚠️  Variable not defined: {e}")
    print("Run the notebook from the beginning or set variables manually:")
    print("# memory_id = 'your-memory-id-here'")
    print("# region = 'us-east-1'")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")